# Construction Crack Detection - Result Analysis

This notebook analyzes the results of the crack detection model on test images and provides insights into the model's performance.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import cv2

# Add project root to path
sys.path.append('..')
from crackdetect.models.segmentation import UNet
from crackdetect.data.dataset import CrackDataset
from crackdetect.utils.crack_analysis import CrackAnalyzer
from crackdetect.inference.predictor import Predictor
from crackdetect.inference.visualization import create_result_figure
from crackdetect.training.metrics import iou_score, dice_coefficient, precision, recall
from config.config import Config

## 1. Load the Trained Model and Test Data

In [ ]:
# Load configuration
config = Config()

# Set paths
model_path = Path("../saved_models/crack_detection_final.pth")
test_dir = Path("../data/test")
results_dir = Path("../results")
results_dir.mkdir(exist_ok=True, parents=True)

# Check if model exists
if not model_path.exists():
    print(f"Model not found at {model_path}. Please train a model first.")
    model_path = list(Path("../saved_models").glob("*.pth"))[0] if list(Path("../saved_models").glob("*.pth")) else None
    if model_path:
        print(f"Using alternative model: {model_path}")
    else:
        print("No model found. Please train a model first.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create predictor
predictor = Predictor(
    model_path=model_path,
    device=device,
    confidence_threshold=0.5,
    pixel_mm_ratio=0.1,  # Adjust as needed
    min_crack_area=100
)

# Load test dataset
test_dataset = CrackDataset(
    image_dir=test_dir / "images",
    mask_dir=test_dir / "masks" if (test_dir / "masks").exists() else None,
    image_size=config.image_size,
    transform=None,
    preprocessing=True
)

print(f"Test dataset size: {len(test_dataset)}")

## 2. Analyze Sample Test Images

In [ ]:
# Choose a sample image
sample_idx = 0
sample_item = test_dataset[sample_idx]
sample_image = sample_item['image'].permute(1, 2, 0).numpy()
sample_filename = sample_item['filename']

# Get ground truth mask if available
has_ground_truth = 'mask' in sample_item
if has_ground_truth:
    sample_mask = sample_item['mask'][0].numpy()

# Analyze image
results = predictor.analyze_image(sample_image)

# Display results
plt.figure(figsize=(18, 12))

plt.subplot(2, 2, 1)
plt.title("Original Image")
plt.imshow(results['processed_image'])
plt.axis('off')

plt.subplot(2, 2, 2)
plt.title("Predicted Mask")
plt.imshow(results['prediction_mask'], cmap='gray')
plt.axis('off')

if has_ground_truth:
    plt.subplot(2, 2, 3)
    plt.title("Ground Truth Mask")
    plt.imshow(sample_mask, cmap='gray')
    plt.axis('off')

plt.subplot(2, 2, 4)
plt.title("Crack Analysis Result")
plt.imshow(results['result_image'])
plt.axis('off')

plt.tight_layout()
plt.savefig(results_dir / f"{sample_filename}_analysis.png", dpi=300)
plt.show()

# Print analysis results
print(f"Analysis Results for {sample_filename}:")
print(f"Number of cracks detected: {len(results['crack_properties'])}")

if results['crack_properties']:
    for i, props in enumerate(results['crack_properties']):
        print(f"\nCrack #{i+1}:")
        print(f"  Severity: {props.severity}")
        print(f"  Average Width: {props.width_avg:.2f} mm")
        print(f"  Maximum Width: {props.width_max:.2f} mm")
        print(f"  Length: {props.length:.2f} mm")
        print(f"  Area: {props.area:.2f} mm²")
        print(f"  Orientation: {props.orientation:.1f}°")

## 3. Evaluate Model Performance

If ground truth masks are available, let's evaluate the model's performance.

In [ ]:
def evaluate_model(dataset, predictor):
    """
    Evaluate model performance on a dataset.
    
    Args:
        dataset: CrackDataset with ground truth masks
        predictor: Predictor instance
        
    Returns:
        DataFrame with evaluation metrics
    """
    # Check if dataset has ground truth masks
    if 'mask' not in dataset[0]:
        print("No ground truth masks available for evaluation.")
        return None
    
    # Initialize metrics
    metrics = {
        'filename': [],
        'iou': [],
        'dice': [],
        'precision': [],
        'recall': [],
        'num_cracks': [],
        'avg_width': [],
        'max_width': [],
        'total_length': []
    }
    
    # Process each image
    for i in tqdm(range(len(dataset)), desc="Evaluating"):
        # Get data
        item = dataset[i]
        image = item['image'].permute(1, 2, 0).numpy()
        mask = item['mask'][0].numpy()
        filename = item['filename']
        
        # Predict
        results = predictor.analyze_image(image)
        pred_mask = results['prediction_mask']
        
        # Calculate metrics
        pred_tensor = torch.tensor(pred_mask).unsqueeze(0).unsqueeze(0)
        mask_tensor = torch.tensor(mask).unsqueeze(0).unsqueeze(0)
        
        iou = iou_score(pred_tensor, mask_tensor).item()
        dice = dice_coefficient(pred_tensor, mask_tensor).item()
        prec = precision(pred_tensor, mask_tensor).item()
        rec = recall(pred_tensor, mask_tensor).item()
        
        # Record metrics
        metrics['filename'].append(filename)
        metrics['iou'].append(iou)
        metrics['dice'].append(dice)
        metrics['precision'].append(prec)
        metrics['recall'].append(rec)
        
        # Record crack properties
        crack_properties = results['crack_properties']
        metrics['num_cracks'].append(len(crack_properties))
        
        if crack_properties:
            widths = [props.width_avg for props in crack_properties]
            max_widths = [props.width_max for props in crack_properties]
            lengths = [props.length for props in crack_properties]
            
            metrics['avg_width'].append(np.mean(widths))
            metrics['max_width'].append(np.max(max_widths))
            metrics['total_length'].append(np.sum(lengths))
        else:
            metrics['avg_width'].append(0)
            metrics['max_width'].append(0)
            metrics['total_length'].append(0)
    
    # Create DataFrame
    df = pd.DataFrame(metrics)
    
    return df

# Only evaluate if ground truth masks are available
if 'mask' in test_dataset[0]:
    eval_df = evaluate_model(test_dataset, predictor)
    
    # Save results
    eval_df.to_csv(results_dir / "evaluation_metrics.csv", index=False)
    
    # Print summary
    print("\nEvaluation Summary:")
    print(f"Average IoU: {eval_df['iou'].mean():.4f}")
    print(f"Average Dice: {eval_df['dice'].mean():.4f}")
    print(f"Average Precision: {eval_df['precision'].mean():.4f}")
    print(f"Average Recall: {eval_df['recall'].mean():.4f}")
    print(f"Average Cracks per Image: {eval_df['num_cracks'].mean():.2f}")
else:
    print("No ground truth masks available for evaluation.")

## 4. Visualize Metrics

In [ ]:
# Only visualize if evaluation was performed
if 'eval_df' in locals():
    # Set up the figure
    plt.figure(figsize=(15, 10))
    
    # IoU distribution
    plt.subplot(2, 2, 1)
    sns.histplot(eval_df['iou'], kde=True)
    plt.title('IoU Distribution')
    plt.xlabel('IoU')
    plt.ylabel('Count')
    plt.axvline(eval_df['iou'].mean(), color='r', linestyle='--', label=f'Mean: {eval_df["iou"].mean():.3f}')
    plt.legend()
    
    # Dice distribution
    plt.subplot(2, 2, 2)
    sns.histplot(eval_df['dice'], kde=True)
    plt.title('Dice Coefficient Distribution')
    plt.xlabel('Dice')
    plt.ylabel('Count')
    plt.axvline(eval_df['dice'].mean(), color='r', linestyle='--', label=f'Mean: {eval_df["dice"].mean():.3f}')
    plt.legend()
    
    # Precision vs. Recall scatter plot
    plt.subplot(2, 2, 3)
    plt.scatter(eval_df['precision'], eval_df['recall'], alpha=0.6)
    plt.title('Precision vs. Recall')
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.grid(True)
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    
    # Number of cracks distribution
    plt.subplot(2, 2, 4)
    sns.countplot(x='num_cracks', data=eval_df)
    plt.title('Number of Cracks per Image')
    plt.xlabel('Number of Cracks')
    plt.ylabel('Count')
    
    plt.tight_layout()
    plt.savefig(results_dir / "metrics_distribution.png", dpi=300)
    plt.show()
    
    # Correlation between metrics
    plt.figure(figsize=(10, 8))
    correlation_metrics = ['iou', 'dice', 'precision', 'recall', 'num_cracks', 'avg_width', 'max_width', 'total_length']
    corr = eval_df[correlation_metrics].corr()
    
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title('Correlation Between Metrics')
    plt.tight_layout()
    plt.savefig(results_dir / "metrics_correlation.png", dpi=300)
    plt.show()
    
    # Boxplot of IoU for different number of cracks
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='num_cracks', y='iou', data=eval_df)
    plt.title('IoU vs. Number of Cracks')
    plt.xlabel('Number of Cracks')
    plt.ylabel('IoU')
    plt.grid(True)
    plt.savefig(results_dir / "iou_vs_cracks.png", dpi=300)
    plt.show()